# Training a Modified LeNet CNN for Breast Cancer Image Classification and deploying on FPGA target, using HLS4ML.
## Vaggelis Ananiadis, Supervisor: Prof. Karakonstantis G.
#### ECE-284 - Processor Design
#### Training code was modified from https://github.com/m3mentomor1/Breast-Cancer-Image-Classification-with-DenseNet121

## Optimized Version

Modified LeNet architecture:
Balasubramaniam S, Velmurugan Y, Jaganathan D, Dhanasekaran S. A Modified LeNet CNN for Breast Cancer Diagnosis in Ultrasound Images. Diagnostics (Basel). 2023;13(17):2746. Published 2023 Aug 24. doi:10.3390/diagnostics13172746

## 1. Import Dependencies

In [58]:
import os
import glob
import pandas as pd             # Pandas
import numpy as np              # NumPy
import matplotlib.pyplot as plt # Matplotlib
import seaborn as sns           # Seaborn
from PIL import Image           # Pillow
import pathlib

# Tensorflow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, optimizers, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

# # Keras
from keras.optimizers import Adam

# hls4ml
import hls4ml
from hls4ml.model.profiling import numerical, get_ymodel_keras

# Custom methods
from callbacks import all_callbacks # Custom callbacks method from hls4ml tutorial
# from custom_plotting import makeRoc # Custom plotting method from hls4ml tutorial

# scikit-learn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

%matplotlib inline
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2023.2/bin:' + os.environ['PATH'] # Vitis Path
os.environ['PATH'] = '/tools/Xilinx/Vivado/2023.2/bin:' + os.environ['PATH'] # Vivado Path

model_dir = "opt_model/" #  Directory to generate tf model.
hls4ml_dir = 'hls4ml_dir' # Directory to generate firmware.
# fpga_target = 'xc7z020clg484-1' # FPGA target (for zedboard: xc7z020clg484-1).
fpga_target = 'XCZU3EG-SBVA484-1-E' # a Zynq UltraScale+ MPSoC
backend = 'Vitis' # Backend to be used (Vitis, Vivado, VivadoAccelerator).
batch_size = 32 # Set the batch size for training.
image_size = (16, 16) # Define the target image size for preprocessing.
num_channels = 1 # Specify the number of color channels in the images (3 for RGB).
image_shape = (image_size[0], image_size[1], num_channels) # Create the image shape tuple based on the specified size and channels.

## 2. Load Dataset.
#### Check dataset_prep.ipynb for data preprocessing.
#### Training dataset was taken from: 
https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset

In [44]:
OUTPUT_PATH = 'dataset'
X_train_val = np.load(os.path.join(OUTPUT_PATH, 'X_train_16.npy'))
X_val       = np.load(os.path.join(OUTPUT_PATH, 'X_val_16.npy'))
X_test      = np.load(os.path.join(OUTPUT_PATH, 'X_test_16.npy'))

y_train_val = np.load(os.path.join(OUTPUT_PATH, 'y_train.npy'))
y_val       = np.load(os.path.join(OUTPUT_PATH, 'y_val.npy'))
y_test      = np.load(os.path.join(OUTPUT_PATH, 'y_test.npy'))
classes     = np.load(os.path.join(OUTPUT_PATH, 'classes.npy'))

## 3. Build Model

In [ ]:
# Model Parameters
num_classes = 3
dropout_rate = 0.2
learning_rate = 0.01
input_shape = image_shape

inputs = layers.Input(shape=input_shape, name='input1')

x = layers.Conv2D(4, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(dropout_rate)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(8, kernel_size=2, strides=2, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(dropout_rate)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(16, kernel_size=4, strides=2, padding='same', activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

## 4. Train Model:

### Train sparse

In [46]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {
    "pruning_schedule": pruning_schedule.ConstantSparsity(
        target_sparsity=0.75,     
        begin_step=31 * 5,        # Start after 5 epochs
        frequency=31              # Prune every epoch
    )
}
model = prune.prune_low_magnitude(model, **pruning_params)

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy

train = True
if train:
    adam = Adam(lr=0.001)

    # loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
    loss_fn = CategoricalCrossentropy()
    model.compile(optimizer=adam, loss=loss_fn, metrics=['accuracy'])
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir=model_dir,
    )
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(
        X_train_val,
        y_train_val,
        batch_size=batch_size,
        epochs=50,
        validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=callbacks.callbacks,
    )
    # Save the model again but with the pruning 'stripped' to use the regular layer types
    model = strip_pruning(model)
    model.save(model_dir + 'KERAS_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model(model_dir + 'KERAS_check_best_model.h5', custom_objects=co, compile=False)
    
# Manually recompile
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

y_keras = model.predict(X_test)
print("\n\nAccuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))

## 5. Evaluate Model

In [ ]:
start_time = time.time()
y_keras = model.predict(X_test)
end_time = time.time()
print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print(f'Inference time: {round((end_time - start_time)*1000, 3)}ms')

## 6. Convert to FPGA firmware w/ hls4ml

In [ ]:
config = hls4ml.utils.config_from_keras_model(model, 
                                              granularity='model', 
                                              default_reuse_factor=16)
                                              # default_precision='fixed<12,2>')

hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config, 
    backend=backend, 
    output_dir=model_dir + hls4ml_dir, 
    part=fpga_target,
    io_type='io_stream')
hls_model.compile()

In [ ]:
hls_model.build(csim=False)

In [61]:
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

In [62]:
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))

Keras  Accuracy: 0.7721518987341772
hls4ml Accuracy: 0.6708860759493671


## 7. Check Vitis HLS reports

In [ ]:
hls4ml.report.read_vivado_report(model_dir + hls4ml_dir)